# Get Evaluation Result

## Evaluator

In [1]:
from check_result import evaluator

full_set = '../full'
test_set = '../test'

evaluator = evaluator(full_set, test_set)

test_list = evaluator.get_random_test_list(200)

In [2]:
import csv

file_all = open('evaluation_average_full.csv', 'w', newline='')
all_full_writer = csv.writer(file_all)
all_full_writer.writerow(["Name","precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

117

In [3]:
file_test = open('evaluation_average_test.csv', 'w', newline='')
all_test_writer = csv.writer(file_test)
all_test_writer.writerow(["Name","precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

117

## Multi-similarity

In [4]:
from retrieve import retrieve
import PIL
import matplotlib.pyplot as plt
import multisimilarity
from torchvision import transforms as T
import csv


model = multisimilarity.BNInception(512)

sz=224
mean=[104, 117, 128]
std=[1, 1, 1]

def MUL(x):
    return x.mul(255)

def trans(im):
    assert im.mode == 'RGB'
    r, g, b = [im.getchannel(i) for i in range(3)]
    # RGB mode also for BGR, 3x8-bit pixels, true color, see PIL doc
    im = PIL.Image.merge('RGB', [b, g, r])
    return im

transform=T.Compose([
        T.Lambda(trans),
        T.Resize((sz,sz)),
        T.ToTensor(),
        T.Lambda(MUL),
        T.Normalize(mean, std),
    ])

r = retrieve(model=model, model_path="models/multisimilarity.pth", embedding_path="models/multiSimilarity.csv", image_path="image_path.csv")
# image = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/yikai/code/Users/yikai.wang/ceshi/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg"


k = [1, 5, 10, 20, 30]
total_precisions_full = [0, 0, 0, 0, 0]
total_recalls_full = [0, 0, 0, 0, 0]
total_precisions_test = [0, 0, 0, 0, 0]
total_recalls_test = [0, 0, 0, 0, 0]

file1 = open('multisimilarity_evaluation_full.csv', 'w', newline='')
writer1 = csv.writer(file1)
writer1.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

file2 = open('multisimilarity_evaluation_test.csv', 'w', newline='')
writer2 = csv.writer(file2)
writer2.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

for image in test_list:
    image = '../full/' + image
    result = r.get_k_image_path(image, transform, k=30)
    pre_full, pre_test = evaluator.get_precision(image, result, k)
    rec_full, rec_test = evaluator.get_recall(image, result, k)
    
    writer1.writerow(pre_full+rec_full)
    writer2.writerow(pre_test+rec_test)
    
    for i in range(5):
        total_precisions_full[i] += pre_full[i]
        total_recalls_full[i] += rec_full[i]
        total_precisions_test[i] += pre_test[i]
        total_recalls_test[i] += rec_test[i]

file1.close()
file2.close()

for i in range(5):
    total_precisions_full[i] = total_precisions_full[i]/200
    total_recalls_full[i] = total_recalls_full[i]/200
    total_precisions_test[i] = total_precisions_test[i]/200
    total_recalls_test[i] = total_recalls_test[i]/200

all_full_writer.writerow(["multisimilarity"]+total_precisions_full+total_recalls_full)
all_test_writer.writerow(["multisimilarity"]+total_precisions_test+total_recalls_test)

print("Finished Evaluation!")
    

Finished Evaluation!


## Triplet

In [5]:
from retrieve import retrieve
import PIL
import matplotlib.pyplot as plt
import triplet
from torchvision import transforms
import csv




mean, std = 0.1307, 0.3081

transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                                                  transforms.Resize((28,28)),
                                                                  transforms.ToTensor(),
                                                                  transforms.Normalize((mean,), (std,))
                                                                  ])

r = retrieve(model=triplet.get_model(), model_path="models/triplet.pth", embedding_path="models/triplet.csv", image_path="image_path.csv")
# image = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/yikai/code/Users/yikai.wang/ceshi/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg"


k = [1, 5, 10, 20, 30]
total_precisions_full = [0, 0, 0, 0, 0]
total_recalls_full = [0, 0, 0, 0, 0]
total_precisions_test = [0, 0, 0, 0, 0]
total_recalls_test = [0, 0, 0, 0, 0]

file1 = open('triplet_evaluation_full.csv', 'w', newline='')
writer1 = csv.writer(file1)
writer1.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

file2 = open('triplet_evaluation_test.csv', 'w', newline='')
writer2 = csv.writer(file2)
writer2.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

for image in test_list:
    image = '../full/' + image
    result = r.get_k_image_path(image, transform, k=30)
    pre_full, pre_test = evaluator.get_precision(image, result, k)
    rec_full, rec_test = evaluator.get_recall(image, result, k)
    
    writer1.writerow(pre_full+rec_full)
    writer2.writerow(pre_test+rec_test)
    
    for i in range(5):
        total_precisions_full[i] += pre_full[i]
        total_recalls_full[i] += rec_full[i]
        total_precisions_test[i] += pre_test[i]
        total_recalls_test[i] += rec_test[i]

file1.close()
file2.close()

for i in range(5):
    total_precisions_full[i] = total_precisions_full[i]/200
    total_recalls_full[i] = total_recalls_full[i]/200
    total_precisions_test[i] = total_precisions_test[i]/200
    total_recalls_test[i] = total_recalls_test[i]/200

all_full_writer.writerow(["triplet"]+total_precisions_full+total_recalls_full)
all_test_writer.writerow(["triplet"]+total_precisions_test+total_recalls_test)

print("Finished Evaluation!")

Finished Evaluation!


## Contrastive

In [6]:
from retrieve import retrieve
import PIL
import matplotlib.pyplot as plt
import contrastive
from torchvision import transforms
import csv




mean, std = 0.1307, 0.3081

transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                                                  transforms.Resize((28,28)),
                                                                  transforms.ToTensor(),
                                                                  transforms.Normalize((mean,), (std,))
                                                                  ])

r = retrieve(model=contrastive.get_model(), model_path="models/contrastive.pth", embedding_path="models/contrastive.csv", image_path="image_path.csv")
# image = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/yikai/code/Users/yikai.wang/ceshi/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg"

k = [1, 5, 10, 20, 30]
total_precisions_full = [0, 0, 0, 0, 0]
total_recalls_full = [0, 0, 0, 0, 0]
total_precisions_test = [0, 0, 0, 0, 0]
total_recalls_test = [0, 0, 0, 0, 0]

file1 = open('contrastive_evaluation_full.csv', 'w', newline='')
writer1 = csv.writer(file1)
writer1.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

file2 = open('contrastive_evaluation_test.csv', 'w', newline='')
writer2 = csv.writer(file2)
writer2.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

for image in test_list:
    image = '../full/' + image
    result = r.get_k_image_path(image, transform, k=30)
    pre_full, pre_test = evaluator.get_precision(image, result, k)
    rec_full, rec_test = evaluator.get_recall(image, result, k)
    
    writer1.writerow(pre_full+rec_full)
    writer2.writerow(pre_test+rec_test)
    
    for i in range(5):
        total_precisions_full[i] += pre_full[i]
        total_recalls_full[i] += rec_full[i]
        total_precisions_test[i] += pre_test[i]
        total_recalls_test[i] += rec_test[i]

file1.close()
file2.close()

for i in range(5):
    total_precisions_full[i] = total_precisions_full[i]/200
    total_recalls_full[i] = total_recalls_full[i]/200
    total_precisions_test[i] = total_precisions_test[i]/200
    total_recalls_test[i] = total_recalls_test[i]/200

all_full_writer.writerow(["contrastive"]+total_precisions_full+total_recalls_full)
all_test_writer.writerow(["contrastive"]+total_precisions_test+total_recalls_test)

print("Finished Evaluation!")

Finished Evaluation!


## cosface

In [7]:
from retrieve import retrieve
import PIL
import matplotlib.pyplot as plt
import cosface
from torchvision import transforms
import csv




transform = transforms.Compose([
            #transforms.RandomHorizontalFlip(),
            transforms.Resize((112,96)),
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
        ])


r = retrieve(model=cosface.get_model(), model_path="models/cosface.pth", embedding_path="models/cosface.csv", image_path="image_path.csv")
# image = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/yikai/code/Users/yikai.wang/ceshi/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg"

k = [1, 5, 10, 20, 30]
total_precisions_full = [0, 0, 0, 0, 0]
total_recalls_full = [0, 0, 0, 0, 0]
total_precisions_test = [0, 0, 0, 0, 0]
total_recalls_test = [0, 0, 0, 0, 0]

file1 = open('cosface_evaluation_full.csv', 'w', newline='')
writer1 = csv.writer(file1)
writer1.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

file2 = open('cosface_evaluation_test.csv', 'w', newline='')
writer2 = csv.writer(file2)
writer2.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

for image in test_list:
    image = '../full/' + image
    result = r.get_k_image_path(image, transform, k=30)
    pre_full, pre_test = evaluator.get_precision(image, result, k)
    rec_full, rec_test = evaluator.get_recall(image, result, k)
    
    writer1.writerow(pre_full+rec_full)
    writer2.writerow(pre_test+rec_test)
    
    for i in range(5):
        total_precisions_full[i] += pre_full[i]
        total_recalls_full[i] += rec_full[i]
        total_precisions_test[i] += pre_test[i]
        total_recalls_test[i] += rec_test[i]

file1.close()
file2.close()

for i in range(5):
    total_precisions_full[i] = total_precisions_full[i]/200
    total_recalls_full[i] = total_recalls_full[i]/200
    total_precisions_test[i] = total_precisions_test[i]/200
    total_recalls_test[i] = total_recalls_test[i]/200

all_full_writer.writerow(["cosface"]+total_precisions_full+total_recalls_full)
all_test_writer.writerow(["cosface"]+total_precisions_test+total_recalls_test)

print("Finished Evaluation!")


loading LResNet model.
Finished Evaluation!


## softtriple

In [8]:
from retrieve import retrieve
import PIL
import matplotlib.pyplot as plt
import softtriple
from torchvision import transforms
import csv

model = softtriple.BNInception(dim=512)

def RGB2BGR(im):
    assert im.mode == 'RGB'
    r, g, b = im.split()
    return PIL.Image.merge('RGB', (b, g, r))

def MUL(x):
    return x.mul(255)
    
normalize = transforms.Normalize(mean=[104., 117., 128.],
                                 std=[1., 1., 1.])

transform=transforms.Compose([
        transforms.Lambda(RGB2BGR),
        transforms.Resize((224,224)),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Lambda(MUL),
        normalize,
    ])

r = retrieve(model=model, model_path="models/softtriple.pth", embedding_path="models/softtriple.csv", image_path="image_path.csv")
# image = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/yikai/code/Users/yikai.wang/ceshi/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg"


k = [1, 5, 10, 20, 30]
total_precisions_full = [0, 0, 0, 0, 0]
total_recalls_full = [0, 0, 0, 0, 0]
total_precisions_test = [0, 0, 0, 0, 0]
total_recalls_test = [0, 0, 0, 0, 0]

file1 = open('softtriple_evaluation_full.csv', 'w', newline='')
writer1 = csv.writer(file1)
writer1.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

file2 = open('softtriple_evaluation_test.csv', 'w', newline='')
writer2 = csv.writer(file2)
writer2.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

for image in test_list:
    image = '../full/' + image
    result = r.get_k_image_path(image, transform, k=30)
    pre_full, pre_test = evaluator.get_precision(image, result, k)
    rec_full, rec_test = evaluator.get_recall(image, result, k)
    
    writer1.writerow(pre_full+rec_full)
    writer2.writerow(pre_test+rec_test)
    
    for i in range(5):
        total_precisions_full[i] += pre_full[i]
        total_recalls_full[i] += rec_full[i]
        total_precisions_test[i] += pre_test[i]
        total_recalls_test[i] += rec_test[i]

file1.close()
file2.close()

for i in range(5):
    total_precisions_full[i] = total_precisions_full[i]/200
    total_recalls_full[i] = total_recalls_full[i]/200
    total_precisions_test[i] = total_precisions_test[i]/200
    total_recalls_test[i] = total_recalls_test[i]/200

all_full_writer.writerow(["softtriple"]+total_precisions_full+total_recalls_full)
all_test_writer.writerow(["softtriple"]+total_precisions_test+total_recalls_test)

print("Finished Evaluation!")

Finished Evaluation!


## arcface

In [9]:
from retrieve import retrieve
import PIL
import matplotlib.pyplot as plt
import arcface
from torchvision import transforms
import csv

mean, std = 0.1307, 0.3081

transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                        transforms.Resize((128,128)),
                                        transforms.ToTensor(),
                                        transforms.Normalize((mean,), (std,))
                                        ])

r = retrieve(model=arcface.get_model(), model_path="models/arcface.pth", embedding_path="models/arcface.csv", image_path="image_path.csv")
# image = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/yikai/code/Users/yikai.wang/ceshi/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg"


k = [1, 5, 10, 20, 30]
total_precisions_full = [0, 0, 0, 0, 0]
total_recalls_full = [0, 0, 0, 0, 0]
total_precisions_test = [0, 0, 0, 0, 0]
total_recalls_test = [0, 0, 0, 0, 0]

file1 = open('arcface_evaluation_full.csv', 'w', newline='')
writer1 = csv.writer(file1)
writer1.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

file2 = open('arcface_evaluation_test.csv', 'w', newline='')
writer2 = csv.writer(file2)
writer2.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

for image in test_list:
    image = '../full/' + image
    result = r.get_k_image_path(image, transform, k=30)
    pre_full, pre_test = evaluator.get_precision(image, result, k)
    rec_full, rec_test = evaluator.get_recall(image, result, k)
    
    writer1.writerow(pre_full+rec_full)
    writer2.writerow(pre_test+rec_test)
    
    for i in range(5):
        total_precisions_full[i] += pre_full[i]
        total_recalls_full[i] += rec_full[i]
        total_precisions_test[i] += pre_test[i]
        total_recalls_test[i] += rec_test[i]

file1.close()
file2.close()

for i in range(5):
    total_precisions_full[i] = total_precisions_full[i]/200
    total_recalls_full[i] = total_recalls_full[i]/200
    total_precisions_test[i] = total_precisions_test[i]/200
    total_recalls_test[i] = total_recalls_test[i]/200

all_full_writer.writerow(["arcface"]+total_precisions_full+total_recalls_full)
all_test_writer.writerow(["arcface"]+total_precisions_test+total_recalls_test)

print("Finished Evaluation!")


Finished Evaluation!


## proxyNCA

In [10]:
from retrieve import retrieve
import PIL
import matplotlib.pyplot as plt
import proxyNCA
from torchvision import transforms
import csv

class Identity(): # used for skipping transforms
    def __call__(self, im):
        return im

class ScaleIntensities():
    def __init__(self, in_range, out_range):
        """ Scales intensities. For example [-1, 1] -> [0, 255]."""
        self.in_range = in_range
        self.out_range = out_range

    def __oldcall__(self, tensor):
        tensor.mul(255)
        return tensor

    def __call__(self, tensor):
        tensor = (
            tensor - self.in_range[0]
        ) / (
            self.in_range[1] - self.in_range[0]
        ) * (
            self.out_range[1] - self.out_range[0]
        ) + self.out_range[0]
        return tensor


class RGBToBGR():
    def __call__(self, im):
        assert im.mode == 'RGB'
        r, g, b = [im.getchannel(i) for i in range(3)]
        # RGB mode also for BGR, 3x8-bit pixels, true color, see PIL doc
        im = PIL.Image.merge('RGB', [b, g, r])
        return im

def make_transform(sz_resize = 256, sz_crop = 227, mean = [104, 117, 128],
        std = [1, 1, 1], rgb_to_bgr = True, is_train = True,
        intensity_scale = [[0, 1],[0, 255]]):
    return transforms.Compose([
        RGBToBGR() if rgb_to_bgr else Identity(),
        transforms.RandomResizedCrop(sz_crop) if is_train else Identity(),
        transforms.Resize(sz_resize) if not is_train else Identity(),
        transforms.CenterCrop(sz_crop) if not is_train else Identity(),
        #transforms.HorizontalFlip() if is_train else Identity(),
        transforms.ToTensor(),
        ScaleIntensities(*intensity_scale) if intensity_scale is not None else Identity(),
        transforms.Normalize(
            mean=mean,
            std=std,
        )
    ])

transform = make_transform()

r = retrieve(model=proxyNCA.get_model(), model_path="models/proxyNCA.pth", embedding_path="models/proxyNCA.csv", image_path="image_path.csv")
# image = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/yikai/code/Users/yikai.wang/ceshi/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg"


k = [1, 5, 10, 20, 30]
total_precisions_full = [0, 0, 0, 0, 0]
total_recalls_full = [0, 0, 0, 0, 0]
total_precisions_test = [0, 0, 0, 0, 0]
total_recalls_test = [0, 0, 0, 0, 0]

file1 = open('proxyNCA_evaluation_full.csv', 'w', newline='')
writer1 = csv.writer(file1)
writer1.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

file2 = open('proxyNCA_evaluation_test.csv', 'w', newline='')
writer2 = csv.writer(file2)
writer2.writerow(["precision@1", "precision@5", "precision@10","precision@20", "precision@30", "recall@1", "recall@5", "recall@10","recall@20", "recall@30"])

for image in test_list:
    image = '../full/' + image
    result = r.get_k_image_path(image, transform, k=30)
    pre_full, pre_test = evaluator.get_precision(image, result, k)
    rec_full, rec_test = evaluator.get_recall(image, result, k)
    
    writer1.writerow(pre_full+rec_full)
    writer2.writerow(pre_test+rec_test)
    
    for i in range(5):
        total_precisions_full[i] += pre_full[i]
        total_recalls_full[i] += rec_full[i]
        total_precisions_test[i] += pre_test[i]
        total_recalls_test[i] += rec_test[i]

file1.close()
file2.close()

for i in range(5):
    total_precisions_full[i] = total_precisions_full[i]/200
    total_recalls_full[i] = total_recalls_full[i]/200
    total_precisions_test[i] = total_precisions_test[i]/200
    total_recalls_test[i] = total_recalls_test[i]/200

all_full_writer.writerow(["proxyNCA"]+total_precisions_full+total_recalls_full)
all_test_writer.writerow(["proxyNCA"]+total_precisions_test+total_recalls_test)

print("Finished Evaluation!")


Finished Evaluation!


In [11]:
file_all.close()
file_test.close()